In [1]:
import h5py
import tensorflow as tf
import numpy as np

from scipy.signal import spectrogram

In [2]:
CLASS_NAMES = ["EQ", "AN"]
NUM_CLASSES = len(CLASS_NAMES)

In [192]:
def get_data(file_path, idx_start, idx_end, idx_slice):
    x_train = None
    y_train = None
    x_test = None
    y_test = None
    with h5py.File(file_path, "r") as f:
        x_train = f["data"][idx_start:idx_slice]
        y_train = f["labels"][idx_start:idx_slice]
        x_test = f["data"][idx_slice:idx_end]
        y_test = f["labels"][idx_slice:idx_end]
        return (x_train, y_train, x_test, y_test)

In [210]:
(x_train, y_train, x_test, y_test) = get_data("/Users/jarek/git/saigon/data/LEN-DB-processed.hdf5", 0, 10000, 8000)

In [211]:
x_train *= 1000000
x_test *= 1000000

In [255]:
X = np.expand_dims(x_train, axis = 2)
X.shape

(8000, 1620, 1)

In [256]:
Y = np.array(y_train)
Y.shape

(8000,)

In [263]:
T = np.expand_dims(x_test, axis = 2)
T.shape

(2000, 1620, 1)

In [264]:
Q = np.array(y_test)
Q.shape

(2000,)

In [257]:
drop_out_rate = 0.1

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1620,1)),
    tf.keras.layers.Conv1D(8, 11, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(16, 7, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(32, 5, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(64, 5, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(128, 3, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [258]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [261]:
model.fit(X, Y, epochs=3)

Epoch 1/3
250/250 [==============================] - 10s 41ms/step - loss: 0.2610 - accuracy: 0.8967
Epoch 2/3
250/250 [==============================] - 10s 40ms/step - loss: 0.2886 - accuracy: 0.9041
Epoch 3/3
250/250 [==============================] - 10s 39ms/step - loss: 0.2692 - accuracy: 0.9115


In [265]:
test_loss, test_acc = model.evaluate(T, Q, verbose=2)

63/63 - 1s - loss: 0.2379 - accuracy: 0.9075
